In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import scipy.optimize as opt
import scipy.special as sp
import slugify.slugify as slugify

In [ ]:
# import data
data_num = 9

raw_data = []

for i in range(data_num):
  raw_data.append(np.loadtxt('data/saxs-202411/S_LK-' + str(i + 1) + '.dat', skiprows=4, max_rows=1150).T)

raw_data = np.array(raw_data)

print(raw_data.shape)

labels = [
  '1gL 0% HCl',
  '1gL 40% HCl',
  '1gL 60% HCl',
  '1gL 80% HCl',
  '1gL 100% HCl',
  '5gL 40% HCl',
  '5gL 60% HCl',
  '5gL 80% HCl',
  '5gL 100% HCl',
]

class_one = [0, 1, 2, 3, 4]

class_two = [5, 6, 7, 8]

class_three = [[1, 5], [2, 6], [3, 7], [4, 8]]

# set numpy print format
np.set_printoptions(precision=2, suppress=True)

# set plot style
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))

In [3]:
# config for plot
x_min = 0.0063
x_max = 0.4
y_min = 0.5


def get_data_in_range(data, plot_title, req=False, x_min_=x_min, x_max_=x_max):
  q_raw = data[0]
  q = q_raw[(q_raw > x_min_) & (q_raw < x_max_)]
  Iq_raw = data[1]
  Iq = Iq_raw[(q_raw > x_min_) & (q_raw < x_max_)]

  lg_q = np.log10(q)
  lg_Iq = np.log10(Iq)

  if req:
    return lg_q, lg_Iq

  plt.figure()
  plt.plot(lg_q, lg_Iq)
  plt.xlabel('log(q)')
  plt.ylabel('log(I)')
  plt.title(plot_title)
  plt.savefig(f'output/saxs/{slugify(plot_title)}.png')
  plt.show()

  return

In [ ]:
# plot zeroth data
plt.figure()
plt.loglog(raw_data[0, 0], raw_data[0, 1], label=labels[0])
plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.title('Zeroth data')
plt.savefig(f'output/saxs/{slugify('Zeroth data')}.png')
plt.legend()
plt.show()

In [ ]:
# plot all data
plt.figure()
for i in range(data_num):
  plt.loglog(raw_data[i, 0], raw_data[i, 1], label=labels[i])

plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.legend()
plt.title('All data')
plt.savefig(f'output/saxs/{slugify("All data")}.png')
plt.show()

In [ ]:
# plot data for calss one
plt.figure()
for i in class_one:
  plt.loglog(raw_data[i, 0], raw_data[i, 1], label=labels[i])

plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.legend()
plt.title('Class One')
plt.savefig(f'output/saxs/{slugify("Class One")}.png')
plt.show()

In [ ]:
# plot data for calss two
plt.figure()
for i in class_two:
  plt.loglog(raw_data[i, 0], raw_data[i, 1], label=labels[i])

plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.legend()
plt.title('Class Two')
plt.savefig(f'output/saxs/{slugify("Class Two")}.png')
plt.show()

In [ ]:
# plt data for class three with subplots
plt.figure(figsize=(10, 10))
fig, axs = plt.subplots(2, 2)
for i, ax in enumerate(axs.flat):
  for j in class_three[i]:
    ax.loglog(raw_data[j, 0], raw_data[j, 1], label=labels[j])

  ax.set_xlabel('q (1/A)')
  ax.set_ylabel('Intensity (a.u.)')
  ax.set_title('Charge density ' + str(i + 1))
  ax.set_xlim(x_min, x_max)
  ax.set_ylim(y_min)
  ax.legend()

plt.tight_layout()
plt.savefig(f'output/saxs/{slugify("Charge Density Class")}.png')
plt.show()

In [ ]:
def get_slopes(data, _breakpoints, plot_title, req=False, show=True, use_exp=True):
  breakpoints = [np.log10(np.exp(i)) for i in _breakpoints] if use_exp else _breakpoints

  q, Iq = get_data_in_range(data, 'data', req=True)

  if show:
    plt.plot(q, Iq)
    plt.xlabel('log(q)')
    plt.ylabel('log(I)')
    plt.title(plot_title)

  slopes = []  # store slopes
  flag = True  # flag for length of breakpoints if less than 2

  # check the length of breakpoints
  if len(breakpoints) < 2:
    print('breakpoints should have at least two elements')
    flag = False

  def label_template(T):
    return f'slope = {T:.2f}'

  # check the first part
  q_ = q[q < breakpoints[0]]
  Iq_ = Iq[q < breakpoints[0]]
  _ = np.polyfit(q_, Iq_, 1)
  p = np.poly1d(_)
  slopes.append(_[0])
  if show:
    plt.plot(q_, p(q_), '-', label=label_template(slopes[0]))

  # check the last part
  q_last = q[q > breakpoints[-1]]
  Iq_last = Iq[q > breakpoints[-1]]
  _last = np.polyfit(q_last, Iq_last, 1)
  p_last = np.poly1d(_last)
  last_slope = _last[0]  # store the last and temporary remove it from the list

  if flag:
    for i in range(len(breakpoints) - 1):
      q_ = q[(q > breakpoints[i]) & (q < breakpoints[i + 1])]
      Iq_ = Iq[(q > breakpoints[i]) & (q < breakpoints[i + 1])]
      _ = np.polyfit(q_, Iq_, 1)
      p = np.poly1d(_)
      slopes.append(_[0])
      if show:
        plt.plot(q_, p(q_), '-', label=label_template(slopes[-1]))

  # plot the last part
  slopes.append(last_slope)

  if show:
    plt.plot(q_last, p_last(q_last), '-', label=label_template(last_slope))

    plt.legend()
    # plt.savefig(f'output/saxs/{slugify(plot_title)}.png')
    plt.show()

  if req:
    return slopes


# get_slopes(raw_data[0], [-4.4, -3.5, -2.6, -1.9, -1.4], plot_title=labels[0])
# get_slopes(raw_data[1], [-4.4, -3.5, -2.6, -1.9, -1.4], plot_title=labels[1])
# get_slopes(raw_data[2], [-3.7, -2.6, -1.9, -1.4], plot_title=labels[2])
# get_slopes(raw_data[3], [-3.7, -2.7, -1.9, -1.4], plot_title=labels[3])
# get_slopes(raw_data[4], [-3.6, -2.7, -1.9], plot_title=labels[4])
# get_slopes(raw_data[5], [-4.4, -3.6, -1.5], plot_title=labels[5])
# get_slopes(raw_data[6], [-3.8, -2.5, -1.5], plot_title=labels[6])
# get_slopes(raw_data[7], [-3.5, -2.8, -1.5], plot_title=labels[7])
# get_slopes(raw_data[8], [-3.3, -2.5, -1.5], plot_title=labels[8])

# ! the range is based on the log plot (not the log10 plot)
all_breakpoints = [
  [-4.4, -3.5, -2.6, -1.9, -1.4],
  [-4.4, -3.5, -2.6, -1.9, -1.4],
  [-3.7, -2.6, -1.9, -1.4],
  [-3.7, -2.7, -1.9, -1.4],
  [-3.6, -2.7, -1.9],
  [-4.4, -3.6, -1.5],
  [-3.8, -2.5, -1.5],
  [-3.5, -2.8, -1.5],
  [-3.3, -2.5, -1.5],
]

slope_data = []  # store all slopes data

for i in range(data_num):
  slope = get_slopes(raw_data[i], all_breakpoints[i], plot_title=labels[i], req=True, show=True)
  print(f'slope for {labels[i]}: {np.array(slope)}')
  slope_data.append(slope)

In [ ]:
slope_data = []

for i in range(data_num):
  slope = get_slopes(raw_data[i], all_breakpoints[i], plot_title=labels[i], req=True, show=False)
  print(f'slope for {labels[i]}: {np.array(slope)}')
  slope_data.append(slope)

In [ ]:
slope_data = []

for i in range(data_num):
  slope = get_slopes(raw_data[i], [-2.0, -1.0], plot_title=labels[i], req=True, show=False, use_exp=False)
  print(f'slope for {labels[i]}: {np.array(slope)}')
  slope_data.append(slope)

In [ ]:
def plot_guinier(data_, point_, title_, formatter_=formatter):
  _range = np.log(data_[0]) < point_

  q = data_[0][_range]
  Iq = data_[1][_range]

  x = q**2
  y = np.log(Iq)

  _ = np.polyfit(x, y, 1)
  p = np.poly1d(_)

  slope = _[0]
  # print(_)

  r2 = np.corrcoef(x, y)[0, 1] ** 2
  # print('r2:', r2)

  Rg = np.sqrt(-slope * 3) / 10  # nm
  print('Rg:', Rg)

  # verify the Rg and q
  print(10 * q[0] * Rg)

  R_for_DLS = Rg * 2 * 1.5
  # print('R_for_DLS:', R_for_DLS)

  plt.scatter(x, y, label=title_, color='r')
  plt.plot(x, p(x), label=f'r2 = {r2:.2f}', color='b')
  plt.title(f'{title_}, R_for_DLS = {R_for_DLS:.2f} nm')
  plt.xlabel('q^2')
  plt.ylabel('log(I)')
  plt.legend()
  plt.gca().xaxis.set_major_formatter(formatter_)
  plt.show()

  pass


plot_guinier(raw_data[1], point_=-4.5, title_=labels[1])

In [ ]:
for i in range(data_num):
  plot_guinier(raw_data[i], point_=-4.5, title_=labels[i])

In [14]:
# set numpy print format
np.set_printoptions(precision=4, suppress=True)

# set plot style
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))


def plot_Iq_q(index_, p0_1=[1.5, 10, 0.2], p0_2=[0.1, 1, 0.1], p0_3=[0.1, -2], point_1=0.16, point_2=0.08, point_3=0.4):
  def Iq_high_q(q_, I_0_, r_p_, S_inc_):
    return I_0_ * 1 / q_ * (2 * sp.j1(q_ * r_p_) / (q_ * r_p_)) ** 2 + S_inc_

  q_ = raw_data[index_, 0]
  Iq_ = raw_data[index_, 1]

  high_q_range_ = raw_data[index_, 0] > point_1
  high_q_popt, high_q_pcov = opt.curve_fit(Iq_high_q, q_[high_q_range_], Iq_[high_q_range_], p0=p0_1)
  # print('high_q_popt:', high_q_popt)
  print(f'I_0: {high_q_popt[0]:.2f}, r_p: {high_q_popt[1]:.2f}, S_inc: {high_q_popt[2]:.2f}')

  def Iq_fit_with_Sq(q_, beta_, k_, xi_):
    return k_ * Iq_high_q(q_, *high_q_popt) / (1 + beta_ * np.exp(-(q_**2) * xi_**2) * Iq_high_q(q_, *high_q_popt))

  fit_range_ = raw_data[index_, 0] > point_2
  fit_popt, fit_pcov = opt.curve_fit(Iq_fit_with_Sq, q_[fit_range_], Iq_[fit_range_], p0=p0_2)
  # print('fit_popt:', fit_popt)
  print(f'beta: {fit_popt[0]:.2f}, k: {fit_popt[1]:.2f}, xi: {fit_popt[2]:.2f}')

  def Iq_pow(q_, k_, m_):
    return Iq_fit_with_Sq(q_, *fit_popt) + k_ * q_**m_

  pow_range_ = raw_data[index_, 0] <= point_3
  bounds_3 = ([0, -np.inf], [1, 0])
  pow_popt, pow_pcov = opt.curve_fit(Iq_pow, q_[pow_range_], Iq_[pow_range_], p0=p0_3, bounds=bounds_3)
  # print('pow_popt:', pow_popt)
  print(f'k: {pow_popt[0]:.2f}, m: {pow_popt[1]:.2f}')

  plt.loglog(q_, Iq_, 'r', label='data')
  plt.loglog(q_, Iq_high_q(q_, *high_q_popt), 'b', label='fit by P(q)')
  plt.loglog(q_, Iq_fit_with_Sq(q_, *fit_popt), 'g', label='fit by P(q) and S(q)')
  plt.loglog(q_, Iq_pow(q_, *pow_popt), 'y', label='fit by P(q) and S(q) and $q^m$')
  plt.title(labels[index_])
  plt.legend()
  plt.xlabel('q')
  plt.ylim(y_min)
  plt.ylabel('I')
  plt.show()

  pass


plot_guinier_config = [
  [0.16, 0.08, 0.05],
  [0.16, 0.08, 0.08],
]

# plot_Iq_q(0)

In [ ]:
for i in range(data_num):
  plot_Iq_q(i)

In [16]:
def P1(q_, L_, b_, f_corr_val_):
  nb = L_ / b_

  def alpha(x_):
    return np.power(1 + (x_ / 3.12) ** 2 + (x_ / 8.67) ** 3, -0.176 / 6)

  def C(nb_):
    if L_ > (10 * b_):
      return 3.06 * np.power(nb_, -0.44)
    else:
      return 1.0

  def w(x_):
    return 0.5 * (1 + np.tanh((x_ - 1.523) / 0.1477))

  u1 = L_ * b_ / 6 * (1 - 3 / (2 * nb) + 3 / (2 * nb**2) - 3 / (4 * nb**3) * (1 - np.exp(-2 * nb))) * q_**2

  u2 = (alpha(nb)) ** 2 * q_**2 * L_ * b_ / 6

  Rg = np.sqrt(L_ * b_ / 6) * alpha(nb)

  return (
    (1 - w(q_ * Rg)) * 2 / u1**2 * (np.exp(-u1) + u1 - 1)
    + f_corr_val_
    * w(q_ * Rg)
    * (
      1.22 * np.power(q_ * Rg, -1 / 0.585)
      + 0.4288 * np.power(q_ * Rg, -2 / 0.585)
      - 1.651 * np.power(q_ * Rg, -3 / 0.585)
    )
    + C(nb) / nb * (4 / 15 + 7 / 15 / u2 - (11 / 15 + 7 / 15 / u2) * np.exp(-u2))
  )


def derivative_P1(q_, L_, b_, f_corr_val_, h_=1e-5):
  return (P1(q_ + h_, L_, b_, f_corr_val_) - P1(q_, L_, b_, f_corr_val_)) / (2 * h_)


def f_corr(q_, L_, b_, f_corr_val_):
  dp1_dq = derivative_P1(q_, L_, b_, f_corr_val_)
  if dp1_dq <= 0:
    return 1
  # else:
  return 0


def compute_f_corr(q_, L_, b_, f_corr_val_):
  return f_corr(q_, L_, b_, f_corr_val_)

In [17]:
def P2(q_, L_, b_):
  pass